# 🔬 IBM-Informed Clustering Experiments

**Based on:** [IBM Clustering Best Practices](https://www.ibm.com/think/topics/clustering)

**Key IBM Insights Applied:**
1. **K-medoids** > K-means for high-dimensional data (768D-2048D embeddings)
2. **Dimensionality reduction critical** - Test multiple PCA levels
3. **No universal best** - Systematic testing required
4. **Soft clustering** (GMM) may outperform hard clustering
5. **Cluster interpretation** as important as metrics

**New Additions:**
- K-medoids (robust to outliers, high-D)
- Multiple PCA levels (50D, 100D, 200D, 300D, 500D)
- Cluster interpretation analysis
- Linkage comparison (Ward, Complete, Average)
- Density-based methods (HDBSCAN)

---
**⚡ Colab + GPU recommended**

## 0. Setup

In [ ]:
# Install packages (including sklearn-extra for K-medoids)
!pip install -q sentence-transformers datasets scikit-learn scikit-learn-extra matplotlib seaborn pandas numpy hdbscan umap-learn

print('✅ All packages installed!')
print('   Note: sklearn-extra provides K-medoids')

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

if device == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('⚠️  GPU strongly recommended!')
    print('   Runtime → Change runtime type → GPU')

## 1. Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import warnings
import json
import time
warnings.filterwarnings('ignore')

# Clustering algorithms
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn_extra.cluster import KMedoids  # IBM-recommended for high-D
import hdbscan

# Metrics
from sklearn.metrics import (
    silhouette_score, 
    silhouette_samples,
    calinski_harabasz_score, 
    davies_bouldin_score
)

# Preprocessing
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.decomposition import PCA

# Embeddings
from sentence_transformers import SentenceTransformer

# Data
from datasets import load_dataset

# Viz
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

np.random.seed(42)

print('✅ Imports complete')
print('   Including K-medoids from sklearn-extra')

## 2. Load Coding Datasets

**Same as before:** SWE-bench + DS-1000 + BigCodeBench + DebugBench

In [ ]:
# Copy load_coding_datasets() function from notebook 03
# For now, placeholder:

print('📥 Loading coding datasets...')
print('⚠️  Copy full load_coding_datasets() from notebook 03')
print('   Then run: questions = load_coding_datasets()')

# Dummy for testing (replace with real data)
questions = [
    {
        'question': f'Code task {i}',
        'language': 'python',
        'domain': ['web', 'data_science', 'api', 'algorithms'][i % 4],
        'task_type': ['bug_fix', 'feature', 'refactor', 'debug'][i % 4],
        'complexity': ['simple', 'medium', 'complex'][i % 3],
        'source': 'dummy'
    }
    for i in range(1000)  # Replace with ~4000 real questions
]

texts = [q['question'] for q in questions]
print(f'✅ Loaded {len(questions)} questions (REPLACE WITH REAL DATA)')

## 3. Extract Embeddings

**Focus:** CodeBERT (proven best) + a few alternatives

In [ ]:
# Test fewer models, focus on quality
embedding_models = {
    'CodeBERT': 'microsoft/codebert-base',           # Proven best
    'GraphCodeBERT': 'microsoft/graphcodebert-base', # Graph-aware variant
    'UniXcoder': 'microsoft/unixcoder-base',         # Unified representation
}

embeddings_dict = {}

for model_name, model_id in embedding_models.items():
    print(f'\nLoading {model_name}...')
    try:
        model = SentenceTransformer(model_id, device=device)
        embeddings = model.encode(
            texts, 
            batch_size=32, 
            show_progress_bar=True, 
            device=device,
            normalize_embeddings=False
        )
        embeddings_dict[model_name] = normalize(embeddings, norm='l2')
        print(f'✅ {embeddings_dict[model_name].shape}')
        
        del model
        if device == 'cuda':
            torch.cuda.empty_cache()
    except Exception as e:
        print(f'❌ {e}')

print(f'\n✅ Extracted {len(embeddings_dict)} embedding sets')

## 4. Dimensionality Reduction (IBM Critical!)

**IBM Insight:** K-means performs poorly on high-D data (768D)

**Test multiple PCA levels:** 50D, 100D, 200D, 300D, 500D

In [ ]:
# Extract metadata for hybrid features
metadata_features = pd.concat([
    pd.get_dummies([q['language'] for q in questions]),
    pd.get_dummies([q['domain'] for q in questions]),
    pd.get_dummies([q['task_type'] for q in questions]),
    pd.get_dummies([q['complexity'] for q in questions])
], axis=1).values
metadata_norm = normalize(metadata_features, norm='l2')

print(f'Metadata features: {metadata_norm.shape}')

# Create comprehensive feature variants
feature_variants = {}
pca_dims = [50, 100, 200, 300, 500]

for model_name, embeddings in embeddings_dict.items():
    # 1. Pure embeddings (baseline)
    feature_variants[f'{model_name}_pure'] = embeddings
    
    # 2. Multiple PCA reductions (IBM recommends for high-D)
    for pca_dim in pca_dims:
        if embeddings.shape[1] >= pca_dim:
            pca = PCA(n_components=pca_dim, random_state=42)
            emb_pca = pca.fit_transform(embeddings)
            emb_pca_norm = normalize(emb_pca, norm='l2')
            feature_variants[f'{model_name}_pca{pca_dim}'] = emb_pca_norm
            print(f'  {model_name} PCA-{pca_dim}D: {emb_pca_norm.shape}')
    
    # 3. Hybrid (embeddings + metadata)
    # Test different weighting
    for emb_weight in [0.7, 0.8, 0.9]:
        meta_weight = 1.0 - emb_weight
        hybrid = np.concatenate([
            embeddings * emb_weight,
            metadata_norm * meta_weight
        ], axis=1)
        hybrid_norm = normalize(hybrid, norm='l2')
        feature_variants[f'{model_name}_hybrid{int(emb_weight*100)}'] = hybrid_norm

print(f'\n✅ Created {len(feature_variants)} feature variants')
print(f'   Pure: {len(embeddings_dict)}')
print(f'   PCA variants: {len([k for k in feature_variants if "pca" in k])}')
print(f'   Hybrid variants: {len([k for k in feature_variants if "hybrid" in k])}')

## 5. Define Clustering Algorithms (IBM-Recommended)

**Key additions:**
- **K-medoids** (IBM: better for high-D, outlier-robust)
- **Multiple linkages** for Agglomerative
- **Soft clustering** (GMM)

In [ ]:
def get_clustering_algorithms(k):
    """Return clustering algorithms for given K."""
    return {
        # Centroid-based (K-means)
        'KMeans': KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300),
        
        # K-medoids (IBM-recommended for high-D)
        'KMedoids': KMedoids(n_clusters=k, metric='cosine', random_state=42, max_iter=300),
        
        # Hierarchical with different linkages
        'Agg_Ward': AgglomerativeClustering(n_clusters=k, linkage='ward'),
        'Agg_Complete': AgglomerativeClustering(n_clusters=k, linkage='complete'),
        'Agg_Average': AgglomerativeClustering(n_clusters=k, linkage='average'),
        
        # Soft clustering (probabilistic)
        'GMM': GaussianMixture(n_components=k, random_state=42, max_iter=300),
        
        # Spectral
        'Spectral': SpectralClustering(n_clusters=k, random_state=42, affinity='nearest_neighbors', n_neighbors=10),
    }

# K values to test (focus on lower K based on results)
k_values = [2, 3, 4, 5, 6, 8, 10, 12, 15, 20]

print('Algorithms to test per K:')
for name in get_clustering_algorithms(2).keys():
    print(f'  - {name}')

print(f'\nK values: {k_values}')
print(f'Total fixed-K experiments: ~{len(feature_variants) * 7 * len(k_values)}')

## 6. Run Experiments

In [ ]:
results = []
total = len(feature_variants) * (7 * len(k_values) + 2)  # +2 for HDBSCAN/DBSCAN

print('='*70)
print(f'RUNNING {total} EXPERIMENTS')
print('='*70)
print('\nThis may take 10-30 minutes...\n')

for feat_idx, (feat_name, features) in enumerate(feature_variants.items()):
    print(f'\n[{feat_idx+1}/{len(feature_variants)}] {feat_name} ({features.shape})')
    
    # Fixed-K algorithms
    for k in k_values:
        algs = get_clustering_algorithms(k)
        
        for alg_name, alg in algs.items():
            try:
                start = time.time()
                
                # Cluster
                if hasattr(alg, 'fit_predict'):
                    labels = alg.fit_predict(features)
                else:
                    alg.fit(features)
                    labels = alg.predict(features)
                
                # Calculate metrics
                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                
                if n_clusters > 1 and n_clusters < len(features):
                    mask = labels >= 0
                    if mask.sum() > n_clusters:
                        sil = silhouette_score(features[mask], labels[mask], metric='cosine')
                        cal = calinski_harabasz_score(features[mask], labels[mask])
                        dav = davies_bouldin_score(features[mask], labels[mask])
                        
                        results.append({
                            'feature': feat_name,
                            'algorithm': alg_name,
                            'k_requested': k,
                            'k_actual': n_clusters,
                            'silhouette': sil,
                            'calinski': cal,
                            'davies': dav,
                            'time_sec': time.time() - start
                        })
            except Exception as e:
                # Silently skip errors
                pass
    
    # HDBSCAN (density-based, auto K)
    try:
        hdb = hdbscan.HDBSCAN(min_cluster_size=50, metric='cosine', min_samples=10)
        labels = hdb.fit_predict(features)
        mask = labels >= 0
        n_clusters = len(set(labels[mask]))
        
        if n_clusters > 1 and mask.sum() > n_clusters:
            sil = silhouette_score(features[mask], labels[mask], metric='cosine')
            results.append({
                'feature': feat_name,
                'algorithm': 'HDBSCAN',
                'k_requested': 'auto',
                'k_actual': n_clusters,
                'silhouette': sil,
                'calinski': np.nan,
                'davies': np.nan,
                'time_sec': 0
            })
    except:
        pass
    
    if (feat_idx + 1) % 5 == 0:
        print(f'  Completed {len(results)} experiments so far...')

print(f'\n{'='*70}')
print(f'✅ COMPLETED {len(results)} EXPERIMENTS')
print(f'{'='*70}')

## 7. Analyze Results

In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('silhouette', ascending=False)

print(f'Total experiments: {len(results_df)}')
print(f'Valid results: {results_df["silhouette"].notna().sum()}')
print(f'\nTop 30 configurations:')
print(results_df.head(30)[['feature', 'algorithm', 'k_actual', 'silhouette', 'calinski', 'davies']])

In [ ]:
best = results_df.iloc[0]

print('='*70)
print('🏆 BEST CONFIGURATION')
print('='*70)
print(f"\nFeature: {best['feature']}")
print(f"Algorithm: {best['algorithm']}")
print(f"K: {best['k_actual']}")
print(f"\nMetrics:")
print(f"  Silhouette: {best['silhouette']:.6f}")
print(f"  Calinski-Harabasz: {best['calinski']:.2f}")
print(f"  Davies-Bouldin: {best['davies']:.4f}")
print(f"  Time: {best['time_sec']:.2f}s")

print(f"\n{'='*70}")
print('COMPARISON')
print(f"{'='*70}")
print(f"  Baseline (K=20, MiniLM): 0.0353")
print(f"  Previous best (CodeBERT K=2 Agg): 0.5501")
print(f"  IBM-informed best: {best['silhouette']:.6f}")
print(f"\nImprovement over baseline: {(best['silhouette'] - 0.0353) / 0.0353 * 100:+.1f}%")
print(f"Improvement over previous: {(best['silhouette'] - 0.5501) / 0.5501 * 100:+.1f}%")

if best['silhouette'] >= 0.4:
    print('\n🎉 ✅ TARGET 0.4+ ACHIEVED!')
else:
    print(f"\n⚠️  Gap to 0.4: {0.4 - best['silhouette']:.4f}")

## 8. IBM-Specific Analysis

**Key Questions:**
1. Does K-medoids beat K-means? (IBM prediction: YES for high-D)
2. Do PCA features beat pure embeddings? (IBM prediction: YES)
3. Does soft clustering (GMM) help?
4. What linkage works best?

In [ ]:
print('='*70)
print('IBM HYPOTHESIS TESTING')
print('='*70)

# 1. K-medoids vs K-means (same features, same K)
print('\n1. K-MEDOIDS vs K-MEANS (high-D robustness)')
print('-'*70)

kmeans_results = results_df[results_df['algorithm'] == 'KMeans']
kmedoids_results = results_df[results_df['algorithm'] == 'KMedoids']

if len(kmeans_results) > 0 and len(kmedoids_results) > 0:
    kmeans_avg = kmeans_results['silhouette'].mean()
    kmedoids_avg = kmedoids_results['silhouette'].mean()
    
    print(f"K-means average: {kmeans_avg:.4f}")
    print(f"K-medoids average: {kmedoids_avg:.4f}")
    
    if kmedoids_avg > kmeans_avg:
        print(f"✅ IBM CORRECT: K-medoids {(kmedoids_avg - kmeans_avg) / kmeans_avg * 100:+.1f}% better")
    else:
        print(f"❌ K-means actually better by {(kmeans_avg - kmedoids_avg) / kmedoids_avg * 100:.1f}%")

# 2. PCA vs Pure embeddings
print('\n2. PCA vs PURE EMBEDDINGS (dimensionality reduction)')
print('-'*70)

pure_results = results_df[results_df['feature'].str.contains('_pure')]
pca_results = results_df[results_df['feature'].str.contains('_pca')]

if len(pure_results) > 0 and len(pca_results) > 0:
    pure_avg = pure_results['silhouette'].mean()
    pca_avg = pca_results['silhouette'].mean()
    
    print(f"Pure embeddings average: {pure_avg:.4f}")
    print(f"PCA-reduced average: {pca_avg:.4f}")
    
    if pca_avg > pure_avg:
        print(f"✅ IBM CORRECT: PCA {(pca_avg - pure_avg) / pure_avg * 100:+.1f}% better")
    else:
        print(f"❌ Pure embeddings actually better by {(pure_avg - pca_avg) / pca_avg * 100:.1f}%")
    
    # Best PCA dimension
    print("\n  Best PCA dimensions:")
    for pca_dim in [50, 100, 200, 300, 500]:
        pca_dim_results = results_df[results_df['feature'].str.contains(f'_pca{pca_dim}')]
        if len(pca_dim_results) > 0:
            avg = pca_dim_results['silhouette'].mean()
            print(f"    PCA-{pca_dim}D: {avg:.4f}")

# 3. GMM (soft) vs K-means (hard)
print('\n3. SOFT (GMM) vs HARD (K-means) CLUSTERING')
print('-'*70)

gmm_results = results_df[results_df['algorithm'] == 'GMM']

if len(kmeans_results) > 0 and len(gmm_results) > 0:
    gmm_avg = gmm_results['silhouette'].mean()
    
    print(f"K-means (hard) average: {kmeans_avg:.4f}")
    print(f"GMM (soft) average: {gmm_avg:.4f}")
    
    if gmm_avg > kmeans_avg:
        print(f"✅ Soft clustering {(gmm_avg - kmeans_avg) / kmeans_avg * 100:+.1f}% better")
    else:
        print(f"❌ Hard clustering better by {(kmeans_avg - gmm_avg) / gmm_avg * 100:.1f}%")

# 4. Linkage comparison
print('\n4. AGGLOMERATIVE LINKAGE COMPARISON')
print('-'*70)

for linkage in ['Ward', 'Complete', 'Average']:
    link_results = results_df[results_df['algorithm'] == f'Agg_{linkage}']
    if len(link_results) > 0:
        avg = link_results['silhouette'].mean()
        best_val = link_results['silhouette'].max()
        print(f"  {linkage}: avg={avg:.4f}, best={best_val:.4f}")

print('\n' + '='*70)

## 9. Cluster Interpretation (IBM: Critical!)

**IBM Insight:** Understanding WHAT clusters represent matters as much as metrics

In [ ]:
# Re-cluster with best config for interpretation
print('='*70)
print('CLUSTER INTERPRETATION (Best Configuration)')
print('='*70)

best_feat_name = best['feature']
best_features = feature_variants[best_feat_name]
best_k = int(best['k_actual'])
best_alg_name = best['algorithm']

# Get algorithm
if best_alg_name == 'KMedoids':
    alg = KMedoids(n_clusters=best_k, metric='cosine', random_state=42)
elif best_alg_name == 'KMeans':
    alg = KMeans(n_clusters=best_k, random_state=42)
elif 'Agg' in best_alg_name:
    linkage = best_alg_name.split('_')[1].lower()
    alg = AgglomerativeClustering(n_clusters=best_k, linkage=linkage)
elif best_alg_name == 'GMM':
    alg = GaussianMixture(n_components=best_k, random_state=42)
else:
    alg = KMeans(n_clusters=best_k, random_state=42)

# Cluster
labels = alg.fit_predict(best_features)

print(f'\nClustering: {best_alg_name} with K={best_k}')
print(f'Features: {best_feat_name}\n')

# Analyze each cluster
for cluster_id in range(best_k):
    mask = labels == cluster_id
    cluster_questions = [questions[i] for i in range(len(questions)) if mask[i]]
    
    print(f'\n{'='*70}')
    print(f'CLUSTER {cluster_id} ({len(cluster_questions)} questions, {len(cluster_questions)/len(questions)*100:.1f}%)')
    print(f'{'='*70}')
    
    # Metadata distribution
    languages = Counter(q['language'] for q in cluster_questions)
    domains = Counter(q['domain'] for q in cluster_questions)
    tasks = Counter(q['task_type'] for q in cluster_questions)
    complexity = Counter(q['complexity'] for q in cluster_questions)
    sources = Counter(q['source'] for q in cluster_questions)
    
    print(f"\nLanguages: {dict(languages.most_common(3))}")
    print(f"Domains: {dict(domains.most_common(3))}")
    print(f"Task types: {dict(tasks.most_common(3))}")
    print(f"Complexity: {dict(complexity)}")
    print(f"Sources: {dict(sources.most_common(3))}")
    
    # Sample questions
    print(f"\nSample questions:")
    for i, q in enumerate(cluster_questions[:3]):
        print(f"  {i+1}. {q['question'][:100]}...")
    
    # Interpretation
    dominant_domain = domains.most_common(1)[0][0]
    dominant_task = tasks.most_common(1)[0][0]
    dominant_complexity = complexity.most_common(1)[0][0]
    
    print(f"\n💡 INTERPRETATION:")
    print(f"   Cluster {cluster_id} appears to be: {dominant_domain} / {dominant_task} / {dominant_complexity}")

## 10. Visualizations

In [ ]:
# Heatmap: K-medoids vs K-means comparison
kmeans_pure = results_df[
    (results_df['algorithm'] == 'KMeans') & 
    (results_df['feature'].str.contains('CodeBERT'))
]
kmedoids_pure = results_df[
    (results_df['algorithm'] == 'KMedoids') & 
    (results_df['feature'].str.contains('CodeBERT'))
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# K-means
if len(kmeans_pure) > 0:
    pivot1 = kmeans_pure.pivot_table(values='silhouette', index='feature', columns='k_actual')
    sns.heatmap(pivot1, annot=True, fmt='.3f', cmap='RdYlGn', center=0.3, ax=ax1)
    ax1.set_title('K-means (CodeBERT)', fontweight='bold')

# K-medoids
if len(kmedoids_pure) > 0:
    pivot2 = kmedoids_pure.pivot_table(values='silhouette', index='feature', columns='k_actual')
    sns.heatmap(pivot2, annot=True, fmt='.3f', cmap='RdYlGn', center=0.3, ax=ax2)
    ax2.set_title('K-medoids (CodeBERT) - IBM Recommended', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# PCA dimension comparison
pca_data = []
for pca_dim in [50, 100, 200, 300, 500]:
    pca_results = results_df[results_df['feature'].str.contains(f'_pca{pca_dim}')]
    if len(pca_results) > 0:
        pca_data.append({
            'dim': pca_dim,
            'avg_silhouette': pca_results['silhouette'].mean(),
            'max_silhouette': pca_results['silhouette'].max()
        })

if len(pca_data) > 0:
    pca_df = pd.DataFrame(pca_data)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(pca_df['dim'], pca_df['avg_silhouette'], marker='o', linewidth=2, label='Average', markersize=10)
    ax.plot(pca_df['dim'], pca_df['max_silhouette'], marker='s', linewidth=2, label='Best', markersize=10)
    ax.axhline(y=0.4, color='green', linestyle='--', linewidth=2, alpha=0.7, label='Target: 0.4')
    ax.set_xlabel('PCA Dimensions', fontsize=12)
    ax.set_ylabel('Silhouette Score', fontsize=12)
    ax.set_title('PCA Dimensionality Impact (IBM: Lower D Better)', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print('No PCA results to visualize')

## 11. Export Results

In [ ]:
# Export comprehensive results
results_df.to_csv('ibm_informed_results.csv', index=False)

# Export best config
best_config = {
    'approach': 'IBM-informed',
    'feature': str(best['feature']),
    'algorithm': str(best['algorithm']),
    'k': int(best['k_actual']),
    'silhouette': float(best['silhouette']),
    'calinski_harabasz': float(best['calinski']),
    'davies_bouldin': float(best['davies']),
    'target_achieved': bool(best['silhouette'] >= 0.4),
    'ibm_recommendations_applied': {
        'k_medoids_tested': True,
        'dimensionality_reduction': True,
        'multiple_pca_levels': True,
        'soft_clustering_tested': True,
        'cluster_interpretation': True
    }
}

with open('ibm_best_config.json', 'w') as f:
    json.dump(best_config, f, indent=2)

print('✅ Exported:')
print('  - ibm_informed_results.csv')
print('  - ibm_best_config.json')

## 12. Final Summary

In [ ]:
print('='*80)
print('IBM-INFORMED CLUSTERING EXPERIMENTS - FINAL SUMMARY')
print('='*80)

print(f'\n📊 EXPERIMENTS:')
print(f'  Total: {len(results_df)}')
print(f'  Feature variants: {len(feature_variants)}')
print(f'  Algorithms: 7 (KMeans, KMedoids, Agg x3, GMM, Spectral)')
print(f'  K values tested: {k_values}')

print(f'\n🏆 BEST CONFIGURATION:')
print(f"  {best['feature']} + {best['algorithm']} (K={int(best['k_actual'])})")
print(f"  Silhouette: {best['silhouette']:.6f}")

print(f'\n📈 IMPROVEMENTS:')
print(f"  vs Baseline (0.0353): {(best['silhouette'] - 0.0353) / 0.0353 * 100:+.1f}%")
print(f"  vs Previous (0.5501): {(best['silhouette'] - 0.5501) / 0.5501 * 100:+.1f}%")

print(f'\n💡 IBM INSIGHTS VALIDATED:')
if len(kmedoids_results) > 0 and kmedoids_avg > kmeans_avg:
    print(f'  ✅ K-medoids beats K-means for high-D data')
if len(pca_results) > 0 and pca_avg > pure_avg:
    print(f'  ✅ Dimensionality reduction improves clustering')
if len(gmm_results) > 0 and gmm_avg > kmeans_avg:
    print(f'  ✅ Soft clustering (GMM) helps')
print(f'  ✅ Cluster interpretation provides actionable insights')

print(f'\n🎯 RECOMMENDATION:')
if best['silhouette'] >= 0.4:
    print(f'  ✅ TARGET ACHIEVED! Deploy this configuration.')
    print(f'  \n  Production config:')
    print(f"    - Embedding: {best['feature'].split('_')[0]}")
    print(f"    - Preprocessing: {best['feature'].split('_')[1]}")
    print(f"    - Algorithm: {best['algorithm']}")
    print(f"    - K: {int(best['k_actual'])}")
else:
    print(f"  ⚠️  Gap to 0.4: {0.4 - best['silhouette']:.4f}")
    print(f'  Consider: More data, domain-specific embeddings, or accept current best')

print(f'\n{'='*80}')